In [16]:
"""!pip install git+https://github.com/openai/whisper.git"""

'!pip install git+https://github.com/openai/whisper.git'

In [1]:
import whisper
import whisper.utils
import os
from pytubefix import YouTube
from pytubefix.cli import on_progress
from moviepy import VideoFileClip
import subprocess
from pysrt import SubRipFile
from deep_translator import GoogleTranslator

In [2]:
folder_path = "./videos"
audio_path = "./audios"
srt_file = "./srt_file"
output_path = "./output"
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
if not os.path.exists(audio_path):
        os.makedirs(audio_path)
if not os.path.exists(output_path):
        os.makedirs(output_path)
if not os.path.exists(srt_file):
        os.makedirs(srt_file)

In [3]:
download = input("Would you like to download a video from youtube? (y/n): ")
if download == "y":
     url = input("Enter the URL of the video you want to download: ")
     yt = YouTube(url, on_progress_callback=on_progress)
     print(yt.title)

     ys = yt.streams.get_highest_resolution()
     ys.download(output_path=folder_path)
else:
     print("ok")

ok


In [4]:
model = whisper.load_model("base")

def Subttl():
    for video in os.listdir(folder_path):
        video_path = os.path.join(folder_path, video)
        audio = os.path.join(audio_path, video.replace(".mp4",".wav"))

        clip = VideoFileClip(video_path)
        clip.audio.write_audiofile(audio)
        clip.close()

        result = model.transcribe(audio,task="transcribe")
        writer = whisper.utils.get_writer("srt", output_dir=srt_file)
        writer(result,video.replace("mp4","srt"))
        subs = SubRipFile.open(srt_file+f"/{video.replace(".mp4",".srt")}",encoding="utf-8")

        def Translate(main,target):
            translator = GoogleTranslator(source=main, target=target)

            for sub in subs:
                sub.text = translator.translate(sub.text)

            subs.save(srt_file+f"/{video.replace(".mp4","_translated.srt")}", encoding='utf-8')

        ask_translate = input(f"Would you like to translate the subtitles for {video}? (y/n): ")

        if ask_translate.lower() == "y":
            ask_main = input("Enter the language of the subtitles(ex: en,fr,de,etc): ")
            ask_target=input("Enter the language you want to translate to?(ex: en,fr,de,etc): ")
            Translate(ask_main,ask_target)
            
        command = ["ffmpeg", "-i", f"videos/{video}", "-vf", f"subtitles=srt_file/{video.replace(".mp4","_translated.srt")}", f"output/{video.replace(".mp4","_subtitled.mp4")}"]
        subprocess.run(command, check=True)

Subttl()

{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'mp42', 'minor_version': '0', 'compatible_brands': 'isommp42', 'creation_time': '2021-06-24T23:45:34.000000Z'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [640, 360], 'bitrate': 425, 'fps': 29.97002997002997, 'codec_name': 'h264', 'profile': '(Main)', 'metadata': {'Metadata': '', 'creation_time': '2021-06-24T23:45:34.000000Z', 'handler_name': 'ISO Media file produced by Google Inc. Created on: 06/24/2021.', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': 'eng', 'default': True, 'fps': 44100, 'bitrate': 128, 'metadata': {'Metadata': '', 'creation_time': '2021-06-24T23:45:34.000000Z', 'handler_name': 'ISO Media file produced by Google Inc. Created on: 06/24/2021.', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 72.68, 'bitrate': 556, 'start': 0.0, 'default

c:\Users\Kanki\Documents\Github\reelslike_video_creation\.venv\Lib\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


MoviePy - Done.
